In [ ]:
!pip install -U ibm-generative-ai
# !pip install -U ibm-watson-machine-learning

In [56]:
#for Preview version in BAM

import dotenv
import os

from genai.credentials import Credentials
from genai.schemas import GenerateParams
from genai.model import Model

dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY", None)
api_endpoint = os.getenv("GENAI_API", None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=1000,
    min_new_tokens=1,
    stop_sequences=['"""']
).dict()

model = Model(model="meta-llama/llama-2-70b-chat",credentials=creds, params=params)
# model = Model(model="codellama/codellama-34b-instruct",credentials=creds, params=params)

/var/folders/nr/1z5zq4yj1cjgn45lr8fydpc00000gn/T/ipykernel_2180/2449578222.py:22: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  ).dict()


In [57]:
#function to extract xml file to string

import xml.etree.ElementTree as et

def extract_xml_file_to_string(xmlfile):
    tree = et.parse(xmlfile)

    root = tree.getroot()
    return et.tostring(root)

In [58]:
#handle long xml file, that navigate the child node of xml and process piece by piece

def extract_long_xml_file_to_string(xmlfile):
    finaloutcome = ""

    prompt_template = '''
    {xml}
    Explain the above xml in one sentense:'''

    for child in root:
        childsoap = et.tostring(child)
        prompt = prompt_template.format(
            xml=childsoap,
        )

        response_text = model.generate([prompt])
        for resp in response_text:
            print(resp.generated_text)
            finaloutcome += resp.generated_text

# use case#1: extract information from mulesoft XML
 
1. Input#1.1 – provide 3 xml examples `'get-chgsap-odata-impl.xml','exp-connext-api-impl.xml','post-servicenow-odata-impl.xml'` export from mulesoft integration project.  – consulting.
2. Input#1.2- need mulesoft SME provide a notes indicate those important information we need to capture from the XML. – consulting.
3. Task - design prompt(s) that input with the XML, and output a text that includes information captured in Name, Value pairs. – ce/watsonx team
4. Task - develop a Jupyter notebook (python code) that load XML, run the LLM and save the output to text file (we name it factsheet). - Ruchi
5. Output - factsheet (json text) – consulting help verify

In [59]:
# Convert the XML to json

def convert_xml_to_json(xmlfile):
    xml = xmlfile

    prompt_template = '''[INST]<<SYS>>convert the xml provide in backquote to json format.\n
    dont show id attributes.\n
    dont show Note at the end.\n
    generate the json directly only.\n
    <<SYS>>\n
    xml:```{xml}```[/INST]\n
    json:'''

    response_text = model.generate([prompt_template.format(
            xml=xml,
        )])

    return response_text[0].generated_text

def convert_xml_to_name_value_pair(xmlfile):
    xml = xmlfile

    prompt_template = '''[INST]<<SYS>>convert the xml provide in backquote to name, value csv format.\n
    dont show id attributes.\n
    generate the csv directly only.\n
    <<SYS>>\n
    xml:```{xml}```[/INST]\n
    csv:'''

    response_text = model.generate([prompt_template.format(
            xml=xml,
        )])

    return response_text[0].generated_text

mulesoftfile = ['get-chgsap-odata-impl.xml','exp-connext-api-impl.xml','post-servicenow-odata-impl.xml']

factsheets = []

for afile in mulesoftfile:
    print(afile)
    xmlstring = extract_xml_file_to_string(afile)
    fact = [convert_xml_to_json(xmlstring[0:4000])]
    print(fact)
    factsheets += [fact]
    print(convert_xml_to_name_value_pair(xmlstring[0:4000]))

get-chgsap-odata-impl.xml
[' {\n      "flow": {\n        "name": "get-chgsap-odata-identifier-flow",\n        "choice": {\n          "when": {\n            "expression": "#[message.attributes.queryParams.\'fromDate\' == null]",\n            "transform": {\n              "message": {\n                "set-payload": "%dw 2.0\\noutput application/json\\nvar spclchar = \'\\\\\'\\\\\'\\n---\\n{\\n\\t\'\\\\$filter\': \'PONUMBER eq \' ++ spclchar ++ message.attributes.queryParams.\'PONumber\'++ spclchar  default \'\',\\n\\t\\n}",\n                "logger": {\n                  "level": "INFO",\n                  "message": "PONumber URL:: #[payload]"\n                }\n              }\n            },\n            "flow-ref": {\n              "name": "get-chgsap-odata-impl"\n            }\n          },\n          "otherwise": {\n            "transform": {\n              "message": {\n                "set-payload": "%dw 2.0\\noutput application/json\\nvar spclchar = \'\\\\\'\\\\\'\\n---\\n{\\n

# Use case#2: build SAP CPI package scaffolder from fact sheet
 
1. Input#2.1 - a factsheet example that include information desirable that mention in Input#1.2 - consulting
2. Input#2.2 - need SAP CPI consultant provide a note indicate those important information we need to fill to the XML files (Mmap, iflw, etc) in the package folder. – consulting
3. Input#2.3 – 3 sample package zip that ready to import to SAP CPI, and well explained in Input#2.2. - consulting
4. Task - design prompt(s) that generate XML files (Mmap, iflw, etc) that to be included in the package folder, which to be import to SAP CPI as base of project. – ce/watsonx team
5. Task - develop a Jupyter notebook (python code) that load the factsheet, run the LLM, save the output to corresponding format file (XML, etc) and place those file to folder structures and finally zip it. - Ruchi
6. Output - package zip file includes XML files (mmap, iflw, etc) in SAP CPI consumable structure and format. – consulting help verify

In [60]:
def generate_mmap(fact):
    prompt_template = '''[INST]
    <<SYS>>
    base on the fact in backquote, generate a mmap file:
    `{fact}`
    <<SYS>>
    [/INST]
    mmap in xml:'''

    response_text = model.generate([prompt_template.format(
            fact=fact,
        )])
    return response_text[0].generated_text

def generate_iflw(fact):
    prompt_template = '''[INST]
    <<SYS>>
    base on the fact in backquote, generate a iflw file:
    `{fact}`
    <<SYS>>
    [/INST]
    iflw in xml:'''

    response_text = model.generate([prompt_template.format(
            fact=fact,
        )])
    return response_text[0].generated_text

for fact in factsheets:
    print(generate_mmap(fact))
    print(generate_iflw(fact))


```
<?xml version="1.0" encoding="UTF-8"?>
<mmap xmlns="http://www.w3.org/1999/xlink"
         xmlns:m="http://www.w3.org/1999/xlink"
         xmlns:f="http://docs.oasis-open.org/odata/ns/2004/10">
  <m:map>
    <m:choice>
      <m:when>
        <m:expression xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:type="m:boolean">
          #[message.attributes.queryParams.'fromDate' == null]
        </m:expression>
        <m:transform>
          <m:message>
            <m:set-payload xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:type="m:string">
              %dw 2.0
              output application/json
              var spclchar = '\\'
              ---
              {
                '$$filter': 'PONUMBER eq ' ++ spclchar ++ message.attributes.queryParams.'PONumber'++ spclchar  default ''
              }
            </m:set-payload>
            <m:logger level="INFO" message="PONumber URL:: #[payload]"/>
          </m:message>
          <m:flow-ref name="get-chgsap

GenAiException: Expecting value: line 1 column 1 (char 0)

# Use case#3 - Generate Technical Spec from SAP CPI
 
1. Input#3.1 - package zip file export from SAP CPI includes XML files (mmap, iflw, etc) that includes information from project. - consulting
2. Input#3.2 - technical spec template in desirable format (e.g. docx/pptx) with placeholder (e.g. [project name]) in the file that ready for substitution. - consulting
3. Input#3.3 - Input#2.2
4. Task - design prompt(s) that extract information from package files (Mmap, iflw, etc). - ce/watsonx team
5. Task - develop a Jupyter notebook (python code) that load/extract the zip file and read the XML files. run the LLM to extract information and substitute them to placeholder in technical spec template. - Ruchi
6. Output - technical spec base on the information provided in package. – consulting help verify

In [61]:
from docx import Document

sapcpifile = ['MM_PIWEB_To_GSAP_PipeLine_R1.mmap']

for afile in sapcpifile:
    print(afile)
    xmlstring = extract_xml_file_to_string(afile)

prompt_template = """[INST]locate the information from the xml provide.\n
extract the information of source interface, target interface, flow, triggers.<<SYS>>
    xml:```{xml}```
    <</SYS>>
    [/INST]
    output:"""

input_file_path =  "tech spec template.docx"

adoc = Document(input_file_path)


response = model.generate([prompt_template.format(xml=xmlstring[0:4000])])
print(response[0].generated_text)


MM_PIWEB_To_GSAP_PipeLine_R1.mmap


* source interface: Pipeline_PIWEB_Source.xsd
* target interface: GSAP_IDOC_TARGET.xsd
* flow:
	+ UsedFuncLib: get_Property.groovy
	+ UsedFuncLib: containsInList.groovy
	+ UsedFuncLib: Convert_UTC_To_CET.groovy
	+ UsedFuncLib: containsInListWithNullCheck.groovy
* triggers: none specified

Note: The information extracted is based on the provided XML snippet. If the entire XML file contains additional information, the output may be incomplete or incorrect.


In [62]:
from docx import Document

xmlstring = """
<xml>
    <interface name="source">Mulesoft API#1</interface>
    <interface name="target">Mulesoft API#2</interface>
    <function name="abc" description="def">
</xml>  
"""

prompt_template = """[INST]locate the information from the xml provide.\n
extract the information of {placeholder}.
show the extract value only.<<SYS>>
    xml:```{xml}```
    <</SYS>>
    [/INST]
    extract value:"""

# input_file_path = "/Users/yingkitw/Desktop/Incubation program deck tw/8 RAG with watsonx.pptx"
input_file_path =  "tech spec template.docx"

adoc = Document(input_file_path)

for paragraph in adoc.paragraphs:
    placeholdertext = paragraph.text
    print(placeholdertext)
    response = model.generate([prompt_template.format(placeholder=placeholdertext, xml=xmlstring[0:4000])])
    paragraph.text = response[0].generated_text
    print(paragraph.text+'->'+response[0].generated_text)

for table in adoc.tables:
    for row in table.rows:
        for cell in row.cells:
            placeholdertext = cell.text
            print(placeholdertext)
            if len(placeholdertext) > 0 and placeholdertext[0] == '[' and placeholdertext[-1] == ']':
                prompttemplate = prompt_template.format(placeholder=placeholdertext,xml=xmlstring[0:4000])
                response = model.generate([prompttemplate])
                cell.text = response[0].generated_text

output_file_path = input_file_path.replace(
        '.docx', '-out.docx')

adoc.save(output_file_path)

Tech spec template


* interface name="source": Mulesoft API#1
* interface name="target": Mulesoft API#2
* function name="abc": def->

* interface name="source": Mulesoft API#1
* interface name="target": Mulesoft API#2
* function name="abc": def



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"
[source interface]
 Mulesoft API#1-> Mulesoft API#1
[target interface]
 Mulesoft API#2-> Mulesoft API#2



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"



* interface name: "source"
* interface name: "target"
* function name: "abc"
* function description: "def"->

* interface name: "source"
* interface name: "target"
* function name: "abc"
* function descripti